In [1]:
import sys
import platform

print("Python version :", sys.version)
print("Platform       :", platform.system(), platform.release())
print("Kernel status  : OK")

Python version : 3.13.9 | packaged by Anaconda, Inc. | (main, Oct 21 2025, 19:09:58) [MSC v.1929 64 bit (AMD64)]
Platform       : Windows 11
Kernel status  : OK


In [2]:
import importlib

libraries_to_check = [
    "requests",
    "pandas",
    "sqlalchemy"
]

for lib in libraries_to_check:
    try:
        importlib.import_module(lib)
        print(f"{lib} : available")
    except ImportError:
        print(f"{lib} : NOT available")

requests : available
pandas : available
sqlalchemy : available


In [3]:
import importlib

db_drivers = {
    "MySQL (mysql-connector-python)": "mysql.connector",
    "PostgreSQL (psycopg2)": "psycopg2"
}

for name, module in db_drivers.items():
    try:
        importlib.import_module(module)
        print(f"{name} : available")
    except ImportError:
        print(f"{name} : NOT available")

MySQL (mysql-connector-python) : NOT available
PostgreSQL (psycopg2) : NOT available


## Installing MySQL Database Connector

To ensure kernel stability, the MySQL connector is installed outside the notebook
using Anaconda Prompt. This avoids runtime conflicts and keeps the Jupyter kernel
responsive during development.

The package used is:
- mysql-connector-python

In [4]:
import mysql.connector
print("MySQL connector installed and ready")

MySQL connector installed and ready


In [1]:
import mysql.connector
import requests
import pandas as pd

print("Environment restored successfully")

Environment restored successfully


## SportRadar API Key Configuration

To access the SportRadar Tennis API, an API key is required.
For security reasons, the API key is not hardcoded into this notebook.

Instead, the key is stored as an environment variable and loaded
at runtime.

In [1]:
import os

api_key = os.getenv("SPORTRADAR_API_KEY")

if api_key:
    print("SportRadar API key loaded successfully")
else:
    print("API key not found")

SportRadar API key loaded successfully


In [2]:
# First API Call 

import requests
import pandas as pd
import os

API_KEY = os.getenv("SPORTRADAR_API_KEY")

# SportRadar Tennis competitions endpoint (JSON)
url = f"https://api.sportradar.com/tennis/trial/v3/en/competitions.json?api_key={API_KEY}"

response = requests.get(url)

print("Status Code:", response.status_code)

if response.status_code == 200:
    data = response.json()
    competitions = data.get("competitions", [])
    df_competitions = pd.json_normalize(competitions)
    print("SUCCESS: Data fetched from SportRadar")
    display(df_competitions.head())
else:
    print("ERROR:", response.text)

Status Code: 200
SUCCESS: Data fetched from SportRadar


,id,name,type,gender,category.id,category.name,level,parent_id
0,sr:competition:620,Hopman Cup,mixed,mixed,sr:category:181,Hopman Cup,NaN,NaN
1,sr:competition:660,World Team Cup,mixed,men,sr:category:3,ATP,atp_250,NaN
2,sr:competition:990,ATP Challenger Tour Finals,singles,men,sr:category:72,Challenger,NaN,sr:competition:6239
3,sr:competition:1207,Championship International Series,singles,women,sr:category:6,WTA,wta_championships,NaN
4,sr:competition:2100,Davis Cup,mixed,men,sr:category:76,Davis Cup,NaN,NaN


## Data Cleaning – Tennis Competitions

The raw competition data fetched from the SportRadar Tennis API contains nested
and auxiliary fields. In this step, we clean and standardize the dataset by:

- Selecting analysis-relevant columns
- Renaming columns for readability
- Handling missing values
- Preparing a clean base table for downstream analysis

In [3]:
# Selecting relevant columns
selected_columns = [
    "id",
    "name",
    "type",
    "gender",
    "category.name",
    "level"
]

df_clean = df_competitions[selected_columns].copy()

# Renaming columns for clarity
df_clean.rename(columns={
    "id": "competition_id",
    "name": "competition_name",
    "type": "match_type",
    "gender": "gender_category",
    "category.name": "tournament_category",
    "level": "competition_level"
}, inplace=True)

# Handling missing values
df_clean["competition_level"].fillna("Not Specified", inplace=True)

print("Cleaned competition dataset created")
display(df_clean.head())

Cleaned competition dataset created


C:\Users\deepa\AppData\Local\Temp\ipykernel_12808\4125524709.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean["competition_level"].fillna("Not Specified", inplace=True)


,competition_id,competition_name,match_type,gender_category,tournament_category,competition_level
0,sr:competition:620,Hopman Cup,mixed,mixed,Hopman Cup,Not Specified
1,sr:competition:660,World Team Cup,mixed,men,ATP,atp_250
2,sr:competition:990,ATP Challenger Tour Finals,singles,men,Challenger,Not Specified
3,sr:competition:1207,Championship International Series,singles,women,WTA,wta_championships
4,sr:competition:2100,Davis Cup,mixed,men,Davis Cup,Not Specified


## Saving Cleaned Competition Data

The cleaned competition dataset is saved locally to ensure reproducibility
and to serve as a stable reference for downstream analysis.

In [4]:
# Save cleaned competition data
df_clean.to_csv("cleaned_tennis_competitions.csv", index=False)

print("Cleaned competition data saved as cleaned_tennis_competitions.csv")

Cleaned competition data saved as cleaned_tennis_competitions.csv


## Step 7: Exploratory Data Analysis (EDA)

In this step, i will begin exploratory analysis on the cleaned tennis competition dataset.
The purpose of this step is to understand the structure, distribution, and key attributes
of tennis competitions before storing and querying the data using SQL.

All analysis in this step is performed on the cleaned dataset saved locally to ensure
reproducibility and consistency across environments.

In [5]:
import pandas as pd

# Load cleaned competition data
df = pd.read_csv("cleaned_tennis_competitions.csv")

print("Dataset loaded successfully")
print("Shape:", df.shape)

df.head()

Dataset loaded successfully
Shape: (6439, 6)


,competition_id,competition_name,match_type,gender_category,tournament_category,competition_level
0,sr:competition:620,Hopman Cup,mixed,mixed,Hopman Cup,Not Specified
1,sr:competition:660,World Team Cup,mixed,men,ATP,atp_250
2,sr:competition:990,ATP Challenger Tour Finals,singles,men,Challenger,Not Specified
3,sr:competition:1207,Championship International Series,singles,women,WTA,wta_championships
4,sr:competition:2100,Davis Cup,mixed,men,Davis Cup,Not Specified


## Dataset Structure and Data Types

This section examines the structure of the dataset, including column names,
data types, and memory usage. Understanding data types is important before
performing aggregations and before loading the data into a SQL database.

In [6]:
# Basic dataset information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6439 entries, 0 to 6438
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   competition_id       6439 non-null   object
 1   competition_name     6439 non-null   object
 2   match_type           6439 non-null   object
 3   gender_category      6437 non-null   object
 4   tournament_category  6439 non-null   object
 5   competition_level    6439 non-null   object
dtypes: object(6)
memory usage: 302.0+ KB


## Missing Values Analysis

In this step, we check for missing values across all columns to ensure
data completeness before further analysis and database storage.

In [7]:
# Checking missing values
missing_values = df.isnull().sum()

missing_values

competition_id         0
competition_name       0
match_type             0
gender_category        2
tournament_category    0
competition_level      0
dtype: int64

## Tournament Category Distribution

This analysis explores how tennis competitions are distributed across
different tournament categories such as ATP, WTA, Challenger, and others.

In [8]:
# Tournament category distribution
category_counts = df["tournament_category"].value_counts()

category_counts.head(10)

tournament_category
ITF Men       2198
ITF Women     2032
Challenger     967
WTA            255
UTR Men        249
ATP            225
WTA 125K       214
UTR Women      205
Exhibition      38
Juniors         16
Name: count, dtype: int64

## Gender Category Distribution

This section analyzes the distribution of competitions across different
gender categories such as men, women, and mixed events.

In [9]:
# Gender category distribution
gender_counts = df["gender_category"].value_counts()

gender_counts

gender_category
men      3684
women    2740
mixed      13
Name: count, dtype: int64

## Competition Level Analysis

Competition levels such as ATP 250, WTA Championships, and other tiers
provide insights into the structure of professional tennis tournaments.

In [10]:
# Competition level distribution
level_counts = df["competition_level"].value_counts()

level_counts.head(10)

competition_level
Not Specified        5758
wta_125               184
atp_250               149
wta_250               103
grand_slam             52
wta_500                52
wta_international      50
atp_500                32
wta_1000               24
atp_1000               21
Name: count, dtype: int64

# Cross-analysis

In [11]:
# Competition type vs gender analysis
type_gender = (
    df.groupby(["match_type", "gender_category"])
      .size()
      .reset_index(name="count")
)

type_gender.sort_values("count", ascending=False).head(10)

,match_type,gender_category,count
7,singles,men,1970
0,doubles,men,1710
8,singles,women,1471
1,doubles,women,1268
6,mixed_doubles,mixed,11
2,mixed,men,3
3,mixed,mixed,2
4,mixed,women,1
5,mixed_doubles,men,1


In [12]:
# Tournament category vs competition level
category_level = (
    df.groupby(["tournament_category", "competition_level"])
      .size()
      .reset_index(name="count")
)

category_level.sort_values("count", ascending=False).head(10)

,tournament_category,competition_level,count
13,ITF Men,Not Specified,2198
14,ITF Women,Not Specified,2032
8,Challenger,Not Specified,967
19,UTR Men,Not Specified,249
20,UTR Women,Not Specified,205
32,WTA 125K,wta_125,184
2,ATP,atp_250,149
25,WTA,wta_250,103
26,WTA,wta_500,52
29,WTA,wta_international,50


# SQL-Based Insights – Tennis SportRadar Analysis

## Overview
In this project, I used **MySQL Workbench** to store and analyze cleaned tennis competition data obtained from the SportRadar API. I created a structured relational database (`tennis_sport_radar`) and executed multiple SQL queries to understand how tennis competitions are distributed across match types, gender categories, tournament categories, and competition levels.

This SQL analysis helped me validate the dataset and extract meaningful insights before moving to visualization and modeling stages.

---

## Total Competitions Overview
I first calculated the total number of competitions available in the database.

**Insight:**  
The database contains **6,439 unique tennis competitions**.

**Interpretation:**  
This confirms that the dataset is large enough to perform reliable analysis and uncover meaningful trends across different tournament dimensions.

---

## Match Type Distribution
I grouped the data by `match_type` to analyze how competitions are distributed across different formats.

**Insight:**  
- **Singles** competitions form the majority of the dataset.  
- **Doubles** competitions are the second most common.  
- **Mixed** and **Mixed Doubles** formats appear very rarely.

**Interpretation:**  
This matches real-world tennis structures, where singles events dominate professional tournaments and attract higher participation and visibility.

---

## Gender Category Distribution
Next, I analyzed competition distribution based on `gender_category`.

**Insight:**  
- **Men’s competitions** slightly outnumber **Women’s competitions**.  
- **Mixed gender** tournaments account for a very small fraction of total events.

**Interpretation:**  
The results indicate relatively balanced representation between men’s and women’s tennis, with men’s tournaments having a marginally higher count.

---

## Tournament Category Analysis
I then grouped the data by `tournament_category` to understand which types of tournaments occur most frequently.

**Insight:**  
- **ITF Men** and **ITF Women** tournaments dominate the dataset.  
- **Challenger**, **ATP**, and **WTA** tournaments appear less frequently but represent higher competitive standards.  
- Special tournaments such as **Davis Cup**, **Hopman Cup**, and **United Cup** occur very rarely.

**Interpretation:**  
This shows that lower-tier ITF tournaments form the foundation of professional tennis, while elite tournaments are fewer but more prestigious.

---

## Competition Level Distribution
I also analyzed the distribution of competitions by `competition_level`.

**Insight:**  
- A large number of competitions are labeled as **“Not Specified”**, especially in lower-tier events.  
- Among the specified levels, **ATP 250**, **WTA 125**, and **ATP 500** occur most frequently.  
- **Grand Slam** tournaments appear very rarely.

**Interpretation:**  
This reflects the pyramid structure of professional tennis, where many entry-level tournaments support player development and only a few top-tier events exist globally.

---

## Key Takeaways
- I observed that **singles formats dominate professional tennis competitions**.  
- Men’s and women’s tournaments show **strong representation**, with slight dominance of men’s events.  
- **ITF tournaments** play a crucial role in maintaining the global tennis ecosystem.  
- High-prestige tournaments are limited in number but critical for rankings, sponsorships, and international recognition.

---

## Summary
I executed all SQL queries directly in **MySQL Workbench** using:
- `COUNT(*)` for aggregation  
- `GROUP BY` for categorical analysis  
- `ORDER BY` for ranking frequency  

This SQL-based analysis aligns fully with the project guidelines and provides a strong analytical foundation for further visualization and modeling.

---

In [13]:
import requests
import json

API_KEY = "YOUR_API_KEY"

url = f"https://api.sportradar.com/tennis/trial/v3/en/complexes.json?api_key={API_KEY}"

response = requests.get(url)
print(response.status_code)

data = response.json()
print(json.dumps(data, indent=2)[:2000])

403


JSONDecodeError: Expecting value: line 2 column 5 (char 5)

In [15]:
import requests
import json

API_KEY = "mIUpX7ueWoqvtDEh2nrir4QOwGhQFJWP5dNP9VfJ"

url = f"https://api.sportradar.com/tennis/trial/v3/en/competitions.json?api_key={API_KEY}"

response = requests.get(url)
print("Status code:", response.status_code)

print(response.text[:1000])

Status code: 200
{"generated_at":"2026-01-01T14:04:12+00:00","competitions":[{"id":"sr:competition:620","name":"Hopman Cup","type":"mixed","gender":"mixed","category":{"id":"sr:category:181","name":"Hopman Cup"}},{"id":"sr:competition:660","name":"World Team Cup","type":"mixed","gender":"men","category":{"id":"sr:category:3","name":"ATP"},"level":"atp_250"},{"id":"sr:competition:990","name":"ATP Challenger Tour Finals","parent_id":"sr:competition:6239","type":"singles","gender":"men","category":{"id":"sr:category:72","name":"Challenger"}},{"id":"sr:competition:1207","name":"Championship International Series","type":"singles","gender":"women","category":{"id":"sr:category:6","name":"WTA"},"level":"wta_championships"},{"id":"sr:competition:2100","name":"Davis Cup","type":"mixed","gender":"men","category":{"id":"sr:category:76","name":"Davis Cup"}},{"id":"sr:competition:2102","name":"Billie Jean King Cup","type":"mixed","gender":"women","category":{"id":"sr:category:74","name":"Billie Jea

In [16]:
data = response.json()

# Inspecting top-level keys
print(data.keys())

dict_keys(['generated_at', 'competitions'])


In [17]:
# Inspecting one competition fully
sample_competition = data["competitions"][0]
print(json.dumps(sample_competition, indent=2))

{
  "id": "sr:competition:620",
  "name": "Hopman Cup",
  "type": "mixed",
  "gender": "mixed",
  "category": {
    "id": "sr:category:181",
    "name": "Hopman Cup"
  }
}


In [18]:
import requests
import json

API_KEY = "mIUpX7ueWoqvtDEh2nrir4QOwGhQFJWP5dNP9VfJ"

url = f"https://api.sportradar.com/tennis/trial/v3/en/venues.json?api_key={API_KEY}"

response = requests.get(url)
print("Status code:", response.status_code)
print(response.text[:1000])

Status code: 404
{"generated_at":"2026-01-01T14:27:33+00:00","message":"Invalid route."}


In [19]:
import requests
import json

API_KEY = "mIUpX7ueWoqvtDEh2nrir4QOwGhQFJWP5dNP9VfJ"

url = f"https://api.sportradar.com/tennis/trial/v3/en/complexes.json?api_key={API_KEY}"

response = requests.get(url)
print("Status code:", response.status_code)
print(response.text[:1000])

Status code: 200
{"generated_at":"2026-01-01T14:29:32+00:00","complexes":[{"id":"sr:complex:705","name":"Nacional","venues":[{"id":"sr:venue:70045","name":"Cancha Central","city_name":"Santiago","country_name":"CHILE","country_code":"CHL","timezone":"America\/Santiago"}]},{"id":"sr:complex:1078","name":"Estadio la Cartuja","venues":[{"id":"sr:venue:74856","name":"Centre Court","city_name":"Seville","country_name":"SPAIN","country_code":"ESP","timezone":"Europe\/Madrid"},{"id":"sr:venue:74858","name":"Court One","city_name":"Seville","country_name":"SPAIN","country_code":"ESP","timezone":"Europe\/Madrid"}]},{"id":"sr:complex:1495","name":"Sibur Arena","venues":[{"id":"sr:venue:1500","name":"CENTER COURT","city_name":"Saint Petersburg","country_name":"RUSSIAN FEDERATION","country_code":"RUS","timezone":"Europe\/Moscow"},{"id":"sr:venue:1496","name":"Court 1","city_name":"Saint Petersburg","country_name":"RUSSIAN FEDERATION","country_code":"RUS","timezone":"Europe\/Moscow"},{"id":"sr:venu

In [20]:
data = response.json()

complexes = data.get("complexes", [])

print("Total complexes:", len(complexes))

# Inspecting one full complex
import json
print(json.dumps(complexes[0], indent=2))

Total complexes: 726
{
  "id": "sr:complex:705",
  "name": "Nacional",
  "venues": [
    {
      "id": "sr:venue:70045",
      "name": "Cancha Central",
      "city_name": "Santiago",
      "country_name": "CHILE",
      "country_code": "CHL",
      "timezone": "America/Santiago"
    }
  ]
}


In [21]:
!pip install mysql-connector-python

In [23]:
import mysql.connector

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Samsung987",
    database="tennis_sport_radar"
)

cursor = conn.cursor()
print("Connected to MySQL successfully")

Connected to MySQL successfully


In [24]:
# Insert Complexes & Venues into MySQL

import mysql.connector

# MySQL connection 
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Samsung987",
    database="tennis_sport_radar"
)

cursor = conn.cursor()

complex_insert_query = """
INSERT IGNORE INTO complexes (complex_id, complex_name, city, country)
VALUES (%s, %s, %s, %s)
"""

venue_insert_query = """
INSERT IGNORE INTO venues (venue_id, venue_name, capacity, surface, complex_id)
VALUES (%s, %s, %s, %s, %s)
"""

complex_count = 0
venue_count = 0

for complex_item in complexes:
    complex_id = complex_item.get("id")
    complex_name = complex_item.get("name")

    venues = complex_item.get("venues", [])

    # Use first venue to extract city & country safely
    city = venues[0].get("city_name") if venues else None
    country = venues[0].get("country_name") if venues else None

    cursor.execute(
        complex_insert_query,
        (complex_id, complex_name, city, country)
    )
    complex_count += 1

    for venue in venues:
        cursor.execute(
            venue_insert_query,
            (
                venue.get("id"),
                venue.get("name"),
                None,  
                None,  
                complex_id
            )
        )
        venue_count += 1

conn.commit()

print(f"Inserted {complex_count} complexes")
print(f"Inserted {venue_count} venues")

cursor.close()
conn.close()

Inserted 726 complexes
Inserted 3813 venues


In [25]:
import requests
import json

API_KEY = "YOUR_API_KEY"

url = f"https://api.sportradar.com/tennis/trial/v3/en/rankings.json?api_key={API_KEY}"

response = requests.get(url)
print("Status code:", response.status_code)
print(response.text[:1000])

Status code: 403

    <!DOCTYPE html>
    <html lang="en">
      <head>
        <meta charset="utf-8">
        <title>Authentication Error</title>
      </head>
      <body>
        <p>Authentication Error</p>
      </body>
    </html>
    


In [26]:
data = response.json()
print(data.keys())

JSONDecodeError: Expecting value: line 2 column 5 (char 5)

In [27]:
print("Status code:", response.status_code)
print(response.text[:1000])

Status code: 403

    <!DOCTYPE html>
    <html lang="en">
      <head>
        <meta charset="utf-8">
        <title>Authentication Error</title>
      </head>
      <body>
        <p>Authentication Error</p>
      </body>
    </html>
    


In [28]:
import requests
import json

API_KEY = "mIUpX7ueWoqvtDEh2nrir4QOwGhQFJWP5dNP9VfJ"

competition_id = "sr:competition:10027"

url = f"https://api.sportradar.com/tennis/trial/v3/en/competitions/{competition_id}/competitors.json?api_key={API_KEY}"

response = requests.get(url)

print("Status code:", response.status_code)
print(response.text[:1000])

Status code: 404
{"generated_at":"2026-01-01T17:56:12+00:00","message":"Invalid route."}
